In [5]:
import pandas as pd
from textblob_de import TextBlobDE as TextBlob
import numpy as np

#read in Twitter data
tweets_raw = pd.read_csv('/Users/stjepankusenic/PycharmProjects/amca/data/raw/Bundestag_Tweets.csv')



,Unnamed: 0,Datetime,Tweet Id,Text,Username,Twitter_Name,Reply_Count,Retweet_Count,Like_Count
0,0,2021-12-24 09:06:37+00:00,1474305500472922113,Ich wünsche Ihnen ein frohes und gesegnetes We...,rbrinkhaus,Ralph Brinkhaus,24.0,5.0,82.0
1,1,2021-12-17 13:41:57+00:00,1471838074413924356,Herzlichen Glückwunsch @_FriedrichMerz. Alles ...,rbrinkhaus,Ralph Brinkhaus,133.0,22.0,611.0
2,2,2021-12-15 12:34:54+00:00,1471096426381942791,@dieLinke Die Rede in voller Länge gibt es in ...,rbrinkhaus,Ralph Brinkhaus,7.0,2.0,13.0
3,3,2021-12-15 11:58:36+00:00,1471087291913449472,Eine Demokratie braucht eine starke #Oppositio...,rbrinkhaus,Ralph Brinkhaus,8.0,1.0,33.0
4,4,2021-12-15 11:58:35+00:00,1471087286301380616,Eine gefährliche Mischung sehen wir in der #Mi...,rbrinkhaus,Ralph Brinkhaus,7.0,1.0,24.0


In [7]:
from tqdm.notebook import tqdm
tqdm.pandas()

import spacy
from spacy.language import Language

# Create emoji matcher
import re
emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

In [9]:
# Create spacy pipeline
pipeline_exclude = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'morphologizer']
nlp_twitter = spacy.load('de_core_news_sm', exclude=pipeline_exclude)
nlp_twitter.Defaults.stop_words |= {"\n    "}

@Language.component("Lemmatize text")
def lemmatize_text(doc):
    doc = [token.lemma_ for token in doc]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Lowercase Text")
def lowercase(doc):
    doc = [token.lower_ for token in doc]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove URLs")
def remove_urls(doc):
    doc = [token.text for token in doc if not token.like_url]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove emojis")
def remove_emojis(doc):
    doc = [token.text for token in doc if not re.match(emoji, token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove mentions")
def remove_mentions(doc):
    doc = [token.text for token in doc if not re.match("@.*", token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove stopwords and punctuation")
def remove_stopwords(doc):
    doc = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return doc

# The add_pipe function appends our functions to the default pipeline.
#nlp_twitter.add_pipe("emoji", first=True)
nlp_twitter.add_pipe("Lemmatize text", name="Lemmatize text", last=True)
nlp_twitter.add_pipe("Lowercase Text", name="Lowercase Text", last=True)
nlp_twitter.add_pipe("Remove URLs", name="Remove URLs", last=True)
nlp_twitter.add_pipe("Remove emojis", name="Remove emojis", last=True)
nlp_twitter.add_pipe("Remove mentions", name="Remove mentions", last=True)
nlp_twitter.add_pipe("Remove stopwords and punctuation", name="Remove stopwords and punctuation", last=True)

<function __main__.remove_stopwords(doc)>

In [ ]:
#loop through all the politicians we want to analyze
data=[]
for name in ['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil 🇪🇺' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas 🇪🇺' , 'Martin Schulz' , 'Dr. Karamba Diaby' , 'Prof. Karl Lauterbach',
            'SteffiLemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin v. Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock (Archiv)',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen, MdB',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , '🇩🇪 Stephan Brandner 🇩🇪' , 'Tino Chrupalla' , 'Götz Frömming, MdB' , 'Leif-Erik Holm']:
    #get tweets from the specific politician and from the desired period
    tweets_analyzing =tweets_raw.loc[tweets_raw['Twitter_Name']==name]
    tweets_analyzing1 =tweets_analyzing.loc[tweets_analyzing['Datetime']>='2017-10-24']
    tweets_analyzing2 =tweets_analyzing1.loc[tweets_analyzing1['Datetime']<='2021-10-26']
    #preprocess the tweets
    tweets =tweets_analyzing2.Text
    test_tweets = tweets.progress_apply(nlp_twitter)
    #create sentiment scores
    preprocess=[]
    for item in test_tweets:
        preprocess.append(' '.join([word for word in item]))

    preprocessed=pd.DataFrame(preprocess)

    blobs=preprocessed[0].apply(TextBlob)
    sentiment=[]
    for blob in blobs:
        sentiment.append(blob.sentiment)
    #get the scores
    polarity=[]
    subjectivity=[]
    for egg in sentiment:
        polarity.append(egg.polarity)
        subjectivity.append(egg.subjectivity)
    #get the means and medians of both values 
    p_mean = np.mean(polarity)
    s_mean = np.mean(subjectivity)

    #get the number of positive, neutral and negative tweets
    positive_p=0
    neutral_p=0
    negative_p=0
    for item_p in polarity:
        if item_p>0:
            positive_p += 1
        elif item_p<0:
            negative_p += 1
        else:
            neutral_p += 1
    #set up list to secure the values generated
    data.append([tweets_analyzing2['Twitter_Name'].iloc[0],p_mean,s_mean,positive_p,neutral_p,negative_p])    

In [ ]:
#set up dataframe with all values
dataf = pd.DataFrame(data, columns=['Name','Polarity_mean','Subjectivity_mean','Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets'])
display(dataf)
dataf.to_csv('sentiment_scores_01.csv')

In [ ]:
#compute the mean for Polarity and sum over number of tweets from the six parties represented
CDU_polarity = np.mean(dataf['Polarity_mean'][0:7])
print('CDU Sentiment: ',CDU_polarity)
FDP_polarity = np.mean(dataf['Polarity_mean'][7:14])
print('FDP Sentiment: ',FDP_polarity)
SPD_polarity = np.mean(dataf['Polarity_mean'][14:21])
print('SPD Sentiment: ',SPD_polarity)
GRUENE_polarity = np.mean(dataf['Polarity_mean'][21:28])
print('GRUENE Sentiment: ',GRUENE_polarity)
LINKE_polarity = np.mean(dataf['Polarity_mean'][28:35])
print('LINKE Sentiment: ',LINKE_polarity)
AFD_polarity = np.mean(dataf['Polarity_mean'][35:42])
print('AFD Sentiment: ',AFD_polarity)
print( )

for category in ['Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets']:
    CDU_cat = np.sum(dataf[category][0:7])
    print('CDU ',category,': ',CDU_cat)
    FDP_cat = np.sum(dataf[category][7:14])
    print('FDP ',category,': ',FDP_cat)
    SPD_cat = np.sum(dataf[category][14:21])
    print('SPD ',category,': ',SPD_cat)
    GRUENE_cat = np.sum(dataf[category][21:28])
    print('GRUENE ',category,': ',GRUENE_cat)
    LINKE_cat = np.sum(dataf[category][28:35])
    print('LINKE ',category,': ',LINKE_cat)
    AFD_cat = np.sum(dataf[category][35:42])
    print('AFD ',category,': ',AFD_cat)
    print( )

In [ ]:
#compute the mean for Polarity and sum over number of tweets for male and female politicians
female_polarity= np.mean(dataf.loc[dataf['Name'].isin(['Nadine Schön' ,'Bettina Stark-Watzinger','Marie-Agnes Strack-Zimmermann','Saskia Esken' ,'SteffiLemke' ,'Katrin Göring-Eckardt' ,'Britta Haßelmann' ,'Annalena Baerbock (Archiv)','Sahra Wagenknecht' ,'Sevim Dağdelen, MdB','Alice Weidel' ,'Beatrix von Storch'])]['Polarity_mean'])
print('Female Sentiment:',female_polarity)
for category in ['Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets']:
    female_cat = np.sum(dataf.loc[dataf['Name'].isin(['Nadine Schön' ,'Bettina Stark-Watzinger','Marie-Agnes Strack-Zimmermann','Saskia Esken' ,'SteffiLemke' ,'Katrin Göring-Eckardt' ,'Britta Haßelmann' ,'Annalena Baerbock (Archiv)','Sahra Wagenknecht' ,'Sevim Dağdelen, MdB','Alice Weidel' ,'Beatrix von Storch'])][category])
    print('Female ',category,': ',female_cat)

male_polarity= np.mean(dataf.loc[~dataf['Name'].isin(['Nadine Schön' ,'Bettina Stark-Watzinger','Marie-Agnes Strack-Zimmermann','Saskia Esken' ,'SteffiLemke' ,'Katrin Göring-Eckardt' ,'Britta Haßelmann' ,'Annalena Baerbock (Archiv)','Sahra Wagenknecht' ,'Sevim Dağdelen, MdB','Alice Weidel' ,'Beatrix von Storch'])]['Polarity_mean'])
print('Male Sentiment:',male_polarity)
for category in ['Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets']:
    male_cat = np.sum(dataf.loc[~dataf['Name'].isin(['Nadine Schön' ,'Bettina Stark-Watzinger','Marie-Agnes Strack-Zimmermann','Saskia Esken' ,'SteffiLemke' ,'Katrin Göring-Eckardt' ,'Britta Haßelmann' ,'Annalena Baerbock (Archiv)','Sahra Wagenknecht' ,'Sevim Dağdelen, MdB','Alice Weidel' ,'Beatrix von Storch'])][category])
    print('Male ',category,': ',male_cat)

In [ ]:
#plot the polarity of tweets over time
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

i=1
for name in ['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil 🇪🇺' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas 🇪🇺' , 'Martin Schulz' , 'Dr. Karamba Diaby' , 'Prof. Karl Lauterbach',
            'SteffiLemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin v. Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock (Archiv)',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen, MdB',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , '🇩🇪 Stephan Brandner 🇩🇪' , 'Tino Chrupalla' , 'Götz Frömming, MdB' , 'Leif-Erik Holm']:
    #get tweets from the specific politician and from the desired period
    tweets_analyzing =tweets_raw.loc[tweets_raw['Twitter_Name']==name]
    tweets_analyzing1 =tweets_analyzing.loc[tweets_analyzing['Datetime']>='2017-10-24']
    tweets_analyzing2 =tweets_analyzing1.loc[tweets_analyzing1['Datetime']<='2021-10-26']
    #preprocess the tweets
    tweets =tweets_analyzing2.Text
    test_tweets = tweets.progress_apply(nlp_twitter)
    #create sentiment scores
    preprocess=[]
    for item in test_tweets:
        preprocess.append(' '.join([word for word in item]))

    preprocessed=pd.DataFrame(preprocess)

    blobs=preprocessed[0].apply(TextBlob)
    sentiment=[]
    for blob in blobs:
        sentiment.append(blob.sentiment)
    #get the scores
    polarity=[]
    for egg in sentiment:
        polarity.append(egg.polarity)
    #get the desired dates and convert timestamp
    tweets_analyzing2['polarity']=polarity
    tweets_analyzing2['Datetime']=pd.to_datetime(tweets_analyzing2['Datetime'])

    dates=tweets_analyzing2['Datetime']
    values=tweets_analyzing2['polarity']

    #smooth the curve, higher sigma means more smoothing
    if len(polarity)>18999:
        s=100
    elif len(polarity)>17999:
        s=95
    elif len(polarity)>16999:
        s=90
    elif len(polarity)>15999:
        s=85
    elif len(polarity)>14999:
        s=80
    elif len(polarity)>13999:
        s=75
    elif len(polarity)>12999:
        s=70
    elif len(polarity)>11999:
        s=65
    elif len(polarity)>10999:
        s=60
    elif len(polarity)>9999:
        s=55
    elif len(polarity)>8999:
        s=50
    elif len(polarity)>7999:
        s=45
    elif len(polarity)>6999:
        s=40
    elif len(polarity)>5999:
        s=35
    elif len(polarity)>4999:
        s=30
    elif len(polarity)>3999:
        s=25
    elif len(polarity)>2999:
        s=20
    elif len(polarity)>1999:
        s=15
    elif len(polarity)>1399:
        s=10
    else:
        s=8
    ysmoothed = gaussian_filter1d(values, sigma=s)

    fig = plt.figure(figsize=(20,200))
    ax = fig.add_subplot(42,1,i)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y'))    
    fig.autofmt_xdate(rotation=45)
    plt.title(name)
    plt.xlabel('Time')
    plt.ylabel('Polarity Score')
    plt.plot(dates, ysmoothed)
    plt.show()
    i +=1